In [1]:
import os
import glob
import numpy as np
import pandas as pd
import csv

In [2]:
#等距抽样
def dataSample(data,sample_count):
    record_count=data.shape[0]
    width=int(record_count/sample_count)#计算抽样间距
    data_sample=[]
    n=0
    for i in range(record_count-1):
        if i%width==0:
            data_sample.append(data.iloc[[i]])
            n +=1
        if  n==sample_count:
            break
        
    data_sample = np.array(data_sample)
    return data_sample

In [3]:
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import metrics
from sklearn.utils import shuffle
from sklearn.model_selection import cross_val_score

In [4]:
def dfcombine(dfh,dfp,df,label,sample_count):
    df=df
    label=label
    dfh=pd.read_csv(dfh)
    dfp=pd.read_csv(dfp)

    print('orginal data')
    print(dfh.shape)
    print(dfp.shape)

    dfh=dfh.drop(labels=['frame'],axis=1)
    dfh=dataSample(dfh,sample_count)
    dfp=dfp.drop(labels=['frame'],axis=1)
    dfp=dataSample(dfp,sample_count)

    print('smaple data')
    print(dfh.shape)
    print(dfp.shape)



    labelH=[]
    labelP=[]
    for i in range(sample_count):
        labelH=np.append(labelH,[0],axis=0)
        labelP=np.append(labelP,[1],axis=0)

    print('label')
    print(labelH.shape)
    print(labelP.shape)
    print('\n')

    df.append([dfh])
    df.append([dfp])
    label.append([labelH])
    label.append([labelP])

    return df,label

In [5]:
df=[]
label=[]
df,label=dfcombine(r'B/DistanceHealth.csv',r'B/DistancePain.csv',df,label,250)
df,label=dfcombine(r'KT/DistanceHealth.csv',r'KT/DistancePain.csv',df,label,250)
df,label=dfcombine(r'MS/DistanceHealth.csv',r'MS/DistancePain.csv',df,label,250)
df,label=dfcombine(r'SW_line/DistanceHealth.csv',r'SW_line/DistancePain.csv',df,label,250)
df,label=dfcombine(r'SW_smooth/DistanceHealth.csv',r'SW_smooth/DistancePain.csv',df,label,250)
df,label=dfcombine(r'TW/DistanceHealth.csv',r'TW/DistancePain.csv',df,label,250)
df=np.array(df)
df=df.reshape(-1,8)
print(df.shape)
label=np.array(label)
label=label.reshape(-1)
print(label.shape)

orginal data
(5503, 9)
(4761, 9)
smaple data
(250, 1, 8)
(250, 1, 8)
label
(250,)
(250,)


orginal data
(1457, 9)
(7240, 9)
smaple data
(250, 1, 8)
(250, 1, 8)
label
(250,)
(250,)


orginal data
(7338, 9)
(8733, 9)
smaple data
(250, 1, 8)
(250, 1, 8)
label
(250,)
(250,)


orginal data
(7555, 9)
(7872, 9)
smaple data
(250, 1, 8)
(250, 1, 8)
label
(250,)
(250,)


orginal data
(4117, 9)
(7653, 9)
smaple data
(250, 1, 8)
(250, 1, 8)
label
(250,)
(250,)


orginal data
(5219, 9)
(4963, 9)
smaple data
(250, 1, 8)
(250, 1, 8)
label
(250,)
(250,)


(3000, 8)
(3000,)


In [6]:
df=pd.DataFrame(df)
df['painlevel'] =label
df

,0,1,2,3,4,5,6,7,painlevel
0,222.991031,368.201032,589.020373,216.113396,337.965975,423.178449,171.230838,328.525494,0.0
1,215.573653,343.023323,502.802148,222.319590,424.571549,387.734445,153.447059,383.631333,0.0
2,195.243438,342.720002,422.427509,238.907932,422.313864,329.946965,143.502613,334.598267,0.0
3,227.721321,428.590714,515.053395,240.010416,489.204456,436.202934,150.791246,286.876280,0.0
4,220.204450,283.001767,467.080293,208.892317,340.587727,309.919344,134.766465,359.402282,0.0
...,...,...,...,...,...,...,...,...,...
2995,235.172277,430.056973,594.899151,258.799150,485.597570,457.389331,182.222392,416.004808,1.0
2996,234.132441,325.161498,560.000893,207.386113,381.921458,400.216192,141.735669,413.696749,1.0
2997,223.986607,359.824957,486.738123,248.394847,463.365946,377.809476,142.863571,400.719353,1.0
2998,196.155551,347.139741,338.514401,233.548710,462.299686,305.006557,98.020406,293.001706,1.0


In [7]:
scores=[]
data=df.drop(labels='painlevel',axis=1).values
target=df.painlevel.values

for j in range(10):

    
    X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=10/30, random_state=10*(j+1))

    ss = StandardScaler()
    X_train_Standar = ss.fit_transform(X_train)
    X_test_Standar = ss.transform(X_test)

    svc = svm.SVC(C=25, kernel='linear',probability=True)
    svc.fit(X_train_Standar,y_train)
    scores.append(svc.score(X_test_Standar, y_test))
    predicted = svc.predict(X_test_Standar)
    
    tT = 0
    tF = 0
    fT = 0
    fF = 0
    for i in range(len(y_test)):
        if (y_test[i] == 0)&(predicted[i] == 0):
            tT += 1
        elif (y_test[i] == 0)&(predicted[i] == 1):
            tF += 1
        elif (y_test[i] == 1)&(predicted[i] == 0):
            fT += 1
        elif (y_test[i] == 1)&(predicted[i] == 1):
            fF += 1
    print(tT,tF,fT,fF)
        
    ##save result
    file = 'result_NO.csv'
    with open(file, 'a+', newline = '') as csvFile:
        csvWriter = csv.writer(csvFile)
        csvWriter.writerow([tT,tF,fT,fF,'N'])    
        
print(scores)
print(sum(scores)/len(scores))

326 159 178 337
338 152 183 327
319 166 168 347
334 164 169 333
323 182 159 336
339 172 156 333
301 178 200 321
314 172 180 334
331 187 161 321
313 181 166 340
[0.663, 0.665, 0.666, 0.667, 0.659, 0.672, 0.622, 0.648, 0.652, 0.653]
0.6567000000000001


In [8]:
df_Standar=ss.transform(df.drop(labels='painlevel',axis=1).values)
pred = svc.predict(df_Standar)
face = (pred==label)
face = face*1
face

array([1, 0, 0, ..., 1, 0, 0])

In [9]:
def dfcombine2(sitaH,sitaP,df,sample_count):
    df=df
    dfh = pd.read_csv(sitaH)
    dfp = pd.read_csv(sitaP)

    print('orginal data')
    print(dfh.shape)
    print(dfp.shape)

    dfh=dfh.drop(labels=['frame'],axis=1)
    dfh=dataSample(dfh,sample_count)
    dfp=dfp.drop(labels=['frame'],axis=1)
    dfp=dataSample(dfp,sample_count)

    print('smaple data')
    print(dfh.shape)
    print(dfp.shape)

    df.append([dfh])
    df.append([dfp])
    return df

In [10]:
df_sita=[]
df_sita=dfcombine2(r'B/sitaHealth.csv',r'B/sitaPain.csv',df_sita,250)
df_sita=dfcombine2(r'KT/sitaHealth.csv',r'KT/sitaPain.csv',df_sita,250)
df_sita=dfcombine2(r'MS/sitaHealth.csv',r'MS/sitaPain.csv',df_sita,250)
df_sita=dfcombine2(r'SW_line/sitaHealth.csv',r'SW_line/sitaPain.csv',df_sita,250)
df_sita=dfcombine2(r'SW_smooth/sitaHealth.csv',r'SW_smooth/sitaPain.csv',df_sita,250)
df_sita=dfcombine2(r'TW/sitaHealth.csv',r'TW/sitaPain.csv',df_sita,250)
df_sita=np.array(df_sita)
df_sita=df_sita.reshape(-1,15)
print('df_sita')
print(df_sita.shape)

orginal data
(5503, 16)
(4761, 16)
smaple data
(250, 1, 15)
(250, 1, 15)
orginal data
(1457, 16)
(7240, 16)
smaple data
(250, 1, 15)
(250, 1, 15)
orginal data
(7338, 16)
(8733, 16)
smaple data
(250, 1, 15)
(250, 1, 15)
orginal data
(7555, 16)
(7872, 16)
smaple data
(250, 1, 15)
(250, 1, 15)
orginal data
(4117, 16)
(7653, 16)
smaple data
(250, 1, 15)
(250, 1, 15)
orginal data
(5219, 16)
(4963, 16)
smaple data
(250, 1, 15)
(250, 1, 15)
df_sita
(3000, 15)


In [11]:
df_sita=pd.DataFrame(df_sita)
df_sita['face'] =face
df_sita

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,face
0,36.498051,79.149447,64.352502,17.171511,128.753684,34.074804,69.184814,64.945033,45.870153,30.277698,34.407984,115.314319,49.604237,51.579495,78.816268,1
1,30.312593,53.428331,96.259076,23.826775,109.571748,46.601478,67.145001,71.865210,40.989789,49.657599,42.961788,87.380613,56.143417,66.788563,57.068021,0
2,27.102011,53.100917,99.797072,26.628476,104.133315,49.238209,54.774135,88.326979,36.898886,50.558863,52.281236,77.159901,51.032397,78.909712,50.057890,0
3,27.736510,61.155655,91.107835,26.047712,96.691741,57.260547,66.824691,75.676833,37.498476,33.847288,56.317676,89.835036,35.536086,82.365388,62.098527,1
4,40.033901,55.759028,84.207071,23.389639,122.643116,33.967245,76.328051,67.183203,36.488745,50.239826,37.251486,92.508688,66.884087,60.641126,52.474787,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,28.958591,62.302004,88.739405,21.073749,114.552929,44.373323,61.588898,75.448480,42.962622,44.366082,46.291099,89.342819,52.250925,67.364847,60.384228,1
2996,37.602573,57.931588,84.465839,20.826234,121.747745,37.426021,81.903253,61.274757,36.821990,47.039818,35.114275,97.845908,63.816157,55.940508,60.243334,1
2997,28.152172,49.283859,102.563969,26.381961,105.069846,48.548193,63.254433,82.024361,34.721206,54.015776,46.602122,79.382102,55.785987,72.984083,51.229931,1
2998,22.862558,43.438348,113.699094,35.004568,81.873825,63.121607,55.889270,99.671019,24.439711,50.577487,66.236462,63.186051,38.435478,101.241030,40.323493,0


In [12]:
import joblib
import time
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier

def SvmRfc(df_sita,FaceModel,SSModel,FaceModelSVM,FaceModelSVMLinear):   
    df1 = df_sita[df_sita.face==0].reset_index(drop=True)
    df2 = df_sita[df_sita.face==1].reset_index(drop=True)
    df2 = pd.DataFrame.sample(df2,n=len(df1),random_state=1).reset_index(drop=True)
    df3 = pd.concat([df1,df2]).reset_index(drop=True) 
    data = df3.drop(labels=['face'],axis=1).values
    target = df3.face.values

    X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.3, random_state=42)


    ss = StandardScaler()
    X_train = ss.fit_transform(X_train)
    X_test = ss.transform(X_test)
    
    ##SVMLinear###########################
    print("svm_linear\n")
    svc_model_linear = svm.SVC(C=25, kernel='linear',probability=True)


    svc_model_linear.fit(X_train, y_train)

    print("score of train",svc_model_linear.score(X_train,y_train))
    
    ##test of svm_linear
    svc_linear_pred = svc_model_linear.predict(X_test)
    print("score of test :",svc_model_linear.score(X_test, y_test))
    print(confusion_matrix(y_test,svc_linear_pred))
    print(classification_report(y_test,svc_linear_pred))
    print("\n")

    ##SVM###############################
    print("svm\n")
    svc_model = svm.SVC(gamma='auto', C=25, kernel='rbf',probability=True)


    svc_model.fit(X_train, y_train)

    print("score of train",svc_model.score(X_train,y_train))

    ##test of svm
    svc_pred = svc_model.predict(X_test)
    print("score of test :",svc_model.score(X_test, y_test))
    print(confusion_matrix(y_test,svc_pred))
    print(classification_report(y_test,svc_pred))
    print("\n")


    
    ##RFC################################
    print("RFC\n")
    rfc = RandomForestClassifier(n_estimators = 40,
     max_depth = 80,
     max_features='auto',
     min_samples_leaf = 1,
     min_samples_split = 4)

    #從訓練組資料中建立隨機森林模型
    rfc.fit(X_train,y_train)
    print("score of train",rfc.score(X_train,y_train))
    
    #test of rfc
    rfc_pred = rfc.predict(X_test)
    print("score of test :",rfc.score(X_test, y_test))
    print(confusion_matrix(y_test,rfc_pred))
    print(classification_report(y_test,rfc_pred))
    
    #保存Model(注:save資料夾要預先建立，否則會錯誤)
    joblib.dump(rfc,FaceModel )
    joblib.dump(svc_model,FaceModelSVM )
    joblib.dump(svc_model_linear,FaceModelSVMLinear )
    joblib.dump(ss,SSModel)
    #svc_model = joblib.load('clf.pkl')

In [13]:
SvmRfc(df_sita,'save/FaceModel_RFC3.pkl','save/ssFace3.pkl','save/FaceModel_SVM3.pkl','save/FaceModel_SVMLinear3.pkl')

svm_linear

score of train 0.5684882895670689
score of test : 0.5487603305785124
[[163 145]
 [128 169]]
              precision    recall  f1-score   support

           0       0.56      0.53      0.54       308
           1       0.54      0.57      0.55       297

    accuracy                           0.55       605
   macro avg       0.55      0.55      0.55       605
weighted avg       0.55      0.55      0.55       605



svm

score of train 0.7771469127040455
score of test : 0.6479338842975206
[[215  93]
 [120 177]]
              precision    recall  f1-score   support

           0       0.64      0.70      0.67       308
           1       0.66      0.60      0.62       297

    accuracy                           0.65       605
   macro avg       0.65      0.65      0.65       605
weighted avg       0.65      0.65      0.65       605



RFC

score of train 0.9985805535841022
score of test : 0.743801652892562
[[231  77]
 [ 78 219]]
              precision    recall  f1-score  

In [14]:
import joblib
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report

def modelPredict2(df_sita,df,faceModel,ssModel,clf):    
    #載入正臉model
    face = joblib.load(faceModel)
    ssFace = joblib.load(ssModel)
    #輸入sita!!!!!!!
    Sita = df_sita
    Sitadata = Sita.drop(labels='face',axis=1).values
    data = ssFace.transform(Sitadata)

    pred = face.predict(data)

    df=df
    df['face']=pred

    df=df[df.face == 1]
    dfh=df[df.painlevel ==0]
    dfp=df[df.painlevel ==1]
    
    #Num of pain = Num of health

    if len(dfp) >= len(dfh):
        dfp=pd.DataFrame.sample(dfp,n=len(dfh),random_state=1).reset_index(drop=True)
        print(len(dfp))
    elif len(dfp) < len(dfh):
        dfh=pd.DataFrame.sample(dfh,n=len(dfp),random_state=1).reset_index(drop=True)
        print(len(dfh))

    # df of train = health+pain
    df = pd.concat([dfh,dfp]).reset_index(drop=True)
    df

    #正規化 > train&test
    data = df.drop(labels=['painlevel','face'],axis=1).values
    target = df.painlevel.values

    
    for j in range(10):
        X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=10/30, random_state=10*(j+1)) 
        ss = StandardScaler()
        X_train = ss.fit_transform(X_train)
        X_test = ss.transform(X_test)

        

        #predict test
        predicted = svc.predict(X_test)

        print("score of test :",svc.score(X_test, y_test))

        print(metrics.classification_report(y_test, predicted))
        print(metrics.confusion_matrix(y_test, predicted))

        tT = 0
        tF = 0
        fT = 0
        fF = 0
        for i in range(len(y_test)):
            if (y_test[i] == 0)&(predicted[i] == 0):
                tT += 1
            elif (y_test[i] == 0)&(predicted[i] == 1):
                tF += 1
            elif (y_test[i] == 1)&(predicted[i] == 0):
                fT += 1
            elif (y_test[i] == 1)&(predicted[i] == 1):
                fF += 1
        print(tT,tF,fT,fF)
        
        ##save result
        file = 'result_NO.csv'
        with open(file, 'a+', newline = '') as csvFile:
            csvWriter = csv.writer(csvFile)
            csvWriter.writerow([tT,tF,fT,fF,clf])

In [15]:
print('svm_line')
modelPredict2(df_sita,df,'save/FaceModel_SVMLinear3.pkl','save/ssFace3.pkl','SVMLinear3')
print('svm')
modelPredict2(df_sita,df,'save/FaceModel_SVM3.pkl','save/ssFace3.pkl','SVM3')
print('rfc')
modelPredict2(df_sita,df,'save/FaceModel_RFC3.pkl','save/ssFace3.pkl','RFC3')

svm_line
666
score of test : 0.6509009009009009
              precision    recall  f1-score   support

         0.0       0.66      0.62      0.64       220
         1.0       0.65      0.68      0.66       224

    accuracy                           0.65       444
   macro avg       0.65      0.65      0.65       444
weighted avg       0.65      0.65      0.65       444

[[137  83]
 [ 72 152]]
137 83 72 152
score of test : 0.6509009009009009
              precision    recall  f1-score   support

         0.0       0.65      0.66      0.66       223
         1.0       0.65      0.64      0.65       221

    accuracy                           0.65       444
   macro avg       0.65      0.65      0.65       444
weighted avg       0.65      0.65      0.65       444

[[148  75]
 [ 80 141]]
148 75 80 141
score of test : 0.6936936936936937
              precision    recall  f1-score   support

         0.0       0.68      0.67      0.68       212
         1.0       0.70      0.72      0.71  

              precision    recall  f1-score   support

         0.0       0.98      0.94      0.96       270
         1.0       0.94      0.98      0.96       277

    accuracy                           0.96       547
   macro avg       0.96      0.96      0.96       547
weighted avg       0.96      0.96      0.96       547

[[253  17]
 [  6 271]]
253 17 6 271
score of test : 0.9414990859232175
              precision    recall  f1-score   support

         0.0       0.94      0.95      0.94       275
         1.0       0.95      0.93      0.94       272

    accuracy                           0.94       547
   macro avg       0.94      0.94      0.94       547
weighted avg       0.94      0.94      0.94       547

[[261  14]
 [ 18 254]]
261 14 18 254
